In [1]:
import time
import datetime
import pytz
import calendar
import itertools
import pandas as pd

In [2]:
# getting the utils file here
import os, sys
sys.path.insert(0, '../')
import utils3 as utils

In [1]:
import datetime
import sys

import networkx as nx
import numpy as np
import os
import pandas as pd
import plotly.offline as py
import pytz

sys.path.insert(0, '../')
# import utils
# from utils import plotly_figure

import itertools
import xbos_services_getter as xsg

# Define OptimizerParent
The optimizer parent can be used accross optimizers. It makes it easy to get the relevant data for training and testing purposes. 

Assumes that an Optimizer instance is created for every time the optimization is called.

In [19]:
class OptimizerParent:
    def __init__(self, building, zones, start, end, window, non_controllable_data=None):
        
        self.start = start
        self.unix_start = start.timestamp() * 1e9
        self.end = end
        self.unix_end = end.timestamp() * 1e9
        self.window = window  # timedelta
        
        self.building = building
        self.zones = zones 
        
        # Documentation: All data here is in timeseries starting exactly at start and every step corresponds to one 
        # interval. The end is not inclusive.
        
        # temperature band
        temperature_band_stub = xsg.get_temperature_band_stub()
        if non_controllable_data is None or non_controllable_data["comfortband"] is None:
            self.comfortband = {iter_zone: xsg.get_comfortband(temperature_band_stub, self.building, iter_zone, self.start, self.end, self.window)
                                  for iter_zone in self.zones}
        if non_controllable_data is None or non_controllable_data["do_not_exceed"] is None:
        self.do_not_exceed = {iter_zone: get_do_not_exceed(temperature_band_stub, self.building, iter_zone, self.start, self.end, self.window)
                              for iter_zone in self.zones}
        
        # occupancy
        occupancy_channel = grpc.insecure_channel(OCCUPANCY_ADDRESS)
        occupancy_stub = occupancy_pb2_grpc.OccupancyStub(occupancy_channel)
        self.occupancy = {iter_zone: get_occupancy(occupancy_stub, self.building, iter_zone, self.start, self.end, self.window)
                         for iter_zone in self.zones}
        
        # outdoor temperatures
        outdoor_historic_channel = grpc.insecure_channel(OUTSIDE_HISTORICAL)
        outdoor_historic_stub = outdoor_temperature_historical_pb2_grpc.OutdoorTemperatureStub(outdoor_historic_channel)
        outdoor_prediction_channel = grpc.insecure_channel(OUTSIDE_PREDICTION)
        outdoor_prediction_stub = outdoor_temperature_prediction_pb2_grpc.OutdoorTemperatureStub(outdoor_prediction_channel)

#         self.outdoor_temperatures = get_outside_temperature(
#             outdoor_historic_stub, outdoor_prediction_stub, self.building, self.start, self.end, self.window)

        # discomfort channel 
        self.discomfort_stub = xsg.get_discomfort_stub()
        
        # HVAC Consumption
        hvac_consumption_stub = xsg.get_hvac_consumption_stub()
        self.hvac_consumption = {iter_zone: get_hvac_consumption(hvac_consumption_stub, building, iter_zone) 
                                 for iter_zone in self.zones}
        
        # TODO Prices
        


# Shortest Path (MPC) Implementation

Here the MPC is implemented. It inherits the data methods from OptimizerParent.

In [112]:

# TODO Demand charges right now assume constant demand charge throughout interval. should be easy to extend
# TODO but need to keep in mind that we need to then store the cost of demand charge and not the consumption
# TODO in the graph, since we actually only want to minimize cost and not consumption.


class Node:
    """
    # this is a Node of the graph for the shortest path
    """

    def __init__(self, temperatures, timestep):
        self.temperatures = temperatures
        self.timestep = timestep

    def __hash__(self):
        return hash((' '.join(str(e) for e in self.temperatures), self.timestep))

    def __eq__(self, other):
        return isinstance(other, self.__class__) \
               and self.temperatures == other.temperatures \
               and self.timestep == other.timestep

    def __repr__(self):
        return "{0}-{1}".format(self.timestep, self.temperatures)


class MPC(OptimizerParent):
    """MPC Optimizer. 
    No Demand Charges and Two Stage actions implemented."""
    
    def __init__(self, building, zones, start, end, window, lambda_val,
                 root=Node([75], 0), non_controllable_data=None, debug=False):
        """
        initialize instance variables
        
        :param building: (str) building name
        :param zones: [str] zone names
        :param start: (datetime timezone aware) 
        :param end: (datetime timezone aware) 
        :param window: (str) the interval in which to split the data.
        :param lambda_val: (float) lambda value for opjective function
        :param root: (Node) the node at which to start

        """
        super().__init__(building, zones, start, end, window, non_controllable_data)

        self.root = root
        self.lambda_val = lambda_val
        self.debug = debug

        self.g = nx.DiGraph()  # [TODO:Changed to MultiDiGraph... FIX print]
        
    def safety_check(self, node):
        for iter_zone in self.zones:
            curr_temperature = node.temperatures[iter_zone]
            curr_safety = self.do_not_exceed[iter_zone].iloc[node.timestep]
            if not (curr_safety["t_low"] <= curr_temperature <= curr_safety["t_high"]):
                return False
        return True
    
    def timestep_to_datetime(self, timestep):
        return start + timestep*datetime.timedelta(seconds=utils.get_window_in_sec(self.window))

    # the shortest path algorithm
    def shortest_path(self, root):
        """
        Creates the graph using DFS and calculates the shortest path
    
        :param root: node being examined right now and needs to be added to graph. 
        
        :return: root Node if root added else return None. 
        
        """
        
        if root is None:
            return None
        
        if root in self.g:
            return root
                
        # stop if node is past predictive horizon
        if self.timestep_to_datetime(root.timestep) >= self.end:
            self.g.add_node(root, objective_cost=0, best_action=None, best_successor=None) # no cost as leaf node
            return root
        
        # check if valid node
        if not self.safety_check(root):
            return None
        
        self.g.add_node(root, objective_cost=np.inf, best_action=None, best_successor=None)

        # creating children, adding corresponding edge and updating root's objective cost
        for action in itertools.product([utils.NO_ACTION, utils.HEATING_ACTION, utils.COOLING_ACTION], 
                                       repeat=len(self.zones)):
                
            # TODO Compute temperatures properly
            temperatures = {}
            for i in range(len(self.zones)):
                 temperatures[self.zones[i]] = root.temperatures[self.zones[i]] + \
                                                    1 * (action[i]==1) - 1*(action[i]==2)

            child_node = Node(
                temperatures=temperatures,
                timestep=root.timestep + 1
                )

            child_node = self.shortest_path(child_node)
            if child_node is None:
                continue

            # get discomfort across edge
            discomfort = {}
            for iter_zone in self.zones:
                curr_comfortband = self.comfortband[iter_zone].iloc[root.timestep]
                curr_occupancy = self.occupancy[iter_zone].iloc[root.timestep]
                average_edge_temperature = (root.temperatures[iter_zone] + child_node.temperatures[iter_zone])/2.

                discomfort[iter_zone] = get_discomfort(
                    self.discomfort_stub, self.building, average_edge_temperature,
                    curr_comfortband["t_low"], curr_comfortband["t_high"], 
                    curr_occupancy)

            # Get consumption across edge
            price = 1  # self.prices.iloc[root.timestep] TODO also add right unit conversion, and duration
            consumption_cost = {self.zones[i]: price * self.hvac_consumption[self.zones[i]][action[i]] 
                               for i in range(len(self.zones))}

            # add edge 
            self.g.add_edge(root, child_node, action=action, discomfort=discomfort, consumption_cost=consumption_cost)

            # update root node to contain the best child.
            total_edge_cost = ((1 - self.lambda_val) * (sum(consumption_cost.values()))) + (
                self.lambda_val * (sum(discomfort.values())))
            
            objective_cost = self.g.node[child_node]["objective_cost"] + total_edge_cost

            if objective_cost < self.g.node[root]["objective_cost"]:
                self.g.node[root]["objective_cost"] = objective_cost
                self.g.node[root]["best_action"] = action
                self.g.node[root]["best_successor"] = child_node
                
        return root
 
    def reconstruct_path(self, root):
        """
        Util function that reconstructs the best action path
        Parameters
        ----------
        graph : networkx graph

        Returns
        -------
        List
        """
        graph = self.g 
        
        if root not in self.g:
            raise Exception("Root does not exist in MPC graph.")

        
        path = [root]

        while graph.node[root]['best_successor'] is not None:
            root = graph.node[root]['best_successor']
            path.append(root)

        return path
    
#     def g_plot(self, zone):
#         try:
#             os.remove('mpc_graph_' + zone + '.html')
#         except OSError:
#             pass

#         fig = plotly_figure(self.advise_unit.g, path=self.path)
#         py.plot(fig, filename='mpc_graph_' + zone + '.html', auto_open=False)


# Preliminary Testing of MPC

In [121]:

end = datetime.datetime.utcnow().replace(tzinfo=pytz.utc)
start =  end - datetime.timedelta(hours=4)
print(start)
print(start.timestamp())
bldg = "avenal-animal-shelter"
zone = "HVAC_Zone_Shelter_Corridor"
window = "15m"

op = MPC(bldg, [zone], start, end, window, 0.995)

2019-02-03 02:16:22.056157+00:00
1549160182.056157


In [122]:
# Run shortest path
root = Node({zone: 85}, 0)
op.occupancy[zone][:] = 1

In [124]:
root = op.shortest_path(root)
print(root)

{'HVAC_Zone_Shelter_Corridor': 0.5}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.5}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.5}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.5}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.5}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.5}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.5}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.5}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor

In [125]:
op.g.node[root]["best_successor"]

1-{'HVAC_Zone_Shelter_Corridor': 84}

In [ ]:
op.safety_check(root)

In [127]:
op.reconstruct_path(root)

[0-{'HVAC_Zone_Shelter_Corridor': 85},
 1-{'HVAC_Zone_Shelter_Corridor': 84},
 2-{'HVAC_Zone_Shelter_Corridor': 83},
 3-{'HVAC_Zone_Shelter_Corridor': 82},
 4-{'HVAC_Zone_Shelter_Corridor': 81},
 5-{'HVAC_Zone_Shelter_Corridor': 80},
 6-{'HVAC_Zone_Shelter_Corridor': 79},
 7-{'HVAC_Zone_Shelter_Corridor': 78},
 8-{'HVAC_Zone_Shelter_Corridor': 78},
 9-{'HVAC_Zone_Shelter_Corridor': 78},
 10-{'HVAC_Zone_Shelter_Corridor': 78},
 11-{'HVAC_Zone_Shelter_Corridor': 78},
 12-{'HVAC_Zone_Shelter_Corridor': 78},
 13-{'HVAC_Zone_Shelter_Corridor': 78},
 14-{'HVAC_Zone_Shelter_Corridor': 78},
 15-{'HVAC_Zone_Shelter_Corridor': 78},
 16-{'HVAC_Zone_Shelter_Corridor': 78}]

In [ ]:
class SimulationMPC():
    
    def __init__(building, zone, start, end, forecasting_horizon, window, indoor_temperature_simulators, non_contrallable_data):
        self.building 
        self.zones
        
        self.start # datetime timezone aware
        self.end # datetime timezone aware
        self.forecasting_horizon # (int seconds)
        self.window # (int seconds, between optimizations)
        self.indoor_temperature_simulators # dictionary of simulator object with key zone. has functions: current_temperature, next_temperature(action)
        
        self.non_controllable_data # dictionary should have keys [outdoor_temperature, occupancy, prices, comfortband, do_not_exceed] and be from start to end + forecasting horizon.
        self.current_time 
        
    def get_time_step(curr_time):
        pass
        
    def next_step():
        
        # call
        MPC(building, zones, start, end, window, lambda_val,
                 root=Node([75], 0), debug=False)
        
        # given the action, update simulation of temperature. 
        # increment time
        
        